Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [ ]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'cuda'

In [ ]:

if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

In [ ]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [ ]:
generate(prompt='Andrej Karpathy, the', num_samples=1, steps=100)

TEST

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)
model.eval();

In [193]:
#embedding vectors of similar words

tokenizer = BPETokenizer()
prompt="A mistake error"
#prompt="A finger finger hand eye eyelash"
#prompt="dog dog dog a A b c d puppy dog cat kitten house explain"
idx=tokenizer(prompt)
print(idx)
wte=model.transformer['wte'].forward(idx)
wtenp=wte.detach().numpy()
wte2=np.array([wtenp[0,i]-wtenp[0,i+1] for i in range(wtenp.shape[1]-1)])
wte2norm=np.linalg.norm(wte2,axis=1)
print(wte2norm)

tensor([[  32, 7457, 4049]])
[4.204302  3.2207382]


In [179]:
out = tokenizer.decode(idx.cpu().squeeze())
print(out)

A finger finger hand eye eyelash


In [ ]:
# Create a 2D tensor
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Swap row 0 and row 2
#tensor[[0, 2]] = tensor[[2, 0]]
print(tensor)

In [180]:
import math
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

from mingpt.utils import CfgNode as CN

In [ ]:
a=3